In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')
sns.set_context('notebook')


%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rcParams['text.latex.preamble'] ='\\usepackage{libertine}\n\\usepackage[utf8]{inputenc}'


from selection import *

import evaluation
from PreEmo import *
from DestiRec import *
import warnings
warnings.filterwarnings("ignore", 'This pattern has match groups')
warnings.filterwarnings("ignore", 'invalid value encountered in true_divide')
warnings.filterwarnings("ignore", 'A class named ')

In [2]:
%load_ext version_information 
%version_information numpy, matplotlib, deap

Software versions
Python 3.9.4 64bit [MSC v.1916 64 bit (AMD64)]
IPython 8.1.1
OS Windows 10 10.0.19041 SP0
numpy 1.22.3
matplotlib 3.5.1
deap 1.3
Sat Apr 23 19:06:33 2022 W. Europe Daylight Time

The Moop Problem

$$
\begin{align*}maximize \qquad  &z_k(x) = \sum_{i=1}^n x_i\ s_{ik}, \hspace{1cm} k = 1,...,p \\
    subject \ to \qquad &\sum_{i=1}^n x_i d_i  \leq D \\
    &\sum_{i=1}^n x_i\ b_i \leq B \\
    &\sum_{j \in N} x_{ij} \geq 1 \\ 
    &g_{il}(x_i) \leq \delta,  \hspace{1cm} \forall \ 1 \leq i < l \leq n  \\
    x_i \in \{0,1\}, \qquad &\forall \ 1 \leq i \leq n
\end{align*}
$$

$$ f^\mathrm{penalty}_i(\mathbf{x}) = \Delta_i - w_i d_i(\mathbf{x}) $$

Possible  Categories

['oct' 'beach' 'watersports' 'entertainment' 'wintersports' 'culture'
 'culinary' 'cities & architecture' 'shopping' 'jan' 'feb' 'mar' 'apr'
 'may' 'jun' 'jul' 'aug' 'sep' 'nov' 'dec' 'hiking' 'nature & wildlife'
 'Safety from crime' 'Weeks to Lower Quantile' 'Weeks to Upper Quantile'
 'average weekly cost']

In [3]:
PreEmo.init_logging('logs/evaluation.log', append=True)


First we load the config file that stores the user input preference, since we do not have a user interface

In [4]:
config = get_config("config.yml")

Get the first input scenario that was used for the evaluation

In [5]:
user_input = config['input']['input1']

We get all the needed dataset using the emo subroutine. 
It loads all the needed data from the sql database, and transforms it into a pandas table. 
It uses user preference information to aggregate the data. 
Start_df is the pandas dataframe. 
The Start_df has also been saved in path directory as 'Pre_Emo_data.csv'.

In [6]:
considered, othercategories, needed_months, months_weeks_dict, start_df  = get_dataset(user_input, rank=False)

In [7]:
start_df.head()

,Region,category,cat_score,childRegions,RId
3154,Afghanistan,beach,1.0,"Afghanistan, Kazakhstan, Kyrgyzstan and Tajiki...",0
3156,Afghanistan,entertainment,2.0,"Afghanistan, Kazakhstan, Kyrgyzstan and Tajiki...",0
3157,Afghanistan,wintersports,1.0,"Afghanistan, Kazakhstan, Kyrgyzstan and Tajiki...",0
3158,Afghanistan,culture,3.0,"Afghanistan, Kazakhstan, Kyrgyzstan and Tajiki...",0
3172,Afghanistan,nov,3.0,"Afghanistan, Kazakhstan, Kyrgyzstan and Tajiki...",0


Get the regions belonging to same branch in a 'region -> regiongroup' dictionary 

In [8]:
region_groups = get_region_groups(start_df)

Get a dictionary of regionname to region Id, for later use in evolutionary process

In [9]:
region_index_info = get_region_index_info(start_df)

Now we can initialize the destination recommender class and pass it the needed parameters

In [10]:
emo = DestiRec(user_input, considered, othercategories, needed_months, months_weeks_dict, start_df, region_groups, region_index_info)


We prepare the toolbox configuration for algorithm variant 5, which is our best performing algorithm

In [11]:
toolbox = emo.prepare_toolbox(NOBJ=4, 
                            POP_SIZE=42, 
                            GEN=120, 
                            P=12, 
                            feasibleOnly=True)

Finally we run the main routine of the evolutionary algorithm

In [12]:
population, stats, result, first_front = emo.main(toolbox)

gen	evals	std	min	avg	maxividual..


Viewing the result inline. Result also stored in result.csv

In [19]:
result['results'] 

[{'EvaluatedRegions': 44,
  'EvaluatedCombinations': 307,
  'Totaltime': 335.65,
  'Input': 0,
  'Variant': 0,
  'Nrun': 0,
  'Score': 31.0,
  'Nregions': 2,
  'SuggestedRegions': 'Indonesia; Vietnam and Cambodia',
  'SuggestedDuration': 'Indonesia; Vietnam and Cambodia-2.0; 1.0',
  'SuggestedBudget': 'Indonesia; Vietnam and Cambodia-400.0; 375.0'}]